In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import pickle
import lightgbm as lgb
from lightgbm import plot_importance
from joblib import Parallel, delayed
from sklearn.model_selection import GroupKFold
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import MaxAbsScaler
import warnings
warnings.filterwarnings('ignore')
seed = 21
np.random.seed(seed)

This notebook is inspired from these very good notebooks and I request everyone to check them out:

[Notebook 1](https://www.kaggle.com/ragnar123/optiver-realized-volatility-lgbm-baseline)

[Notebook 2](https://www.kaggle.com/somang1418/tuning-hyperparameters-under-10-minutes-lgbm)

[Notebook 3](https://www.kaggle.com/felipefonte99/optiver-lgb-with-optimized-params)

I have optimized the code and tried to make it more beginner friendly (hence for myself)

In [ ]:
base_dir = "../input/optiver-realized-volatility-prediction/"

# Model Building

In [ ]:
# Function to calculate wap
def calc_wap(bid_price, ask_price, bid_size, ask_size):
    wap = ((bid_price*ask_size) + (ask_price*bid_size))/(bid_size+ask_size)
    return wap


# Function to calculate log return
def calc_lr(wap_series):
    return np.log(wap_series).diff()


# Function to calculate realized volatility
def realized_volatility(log_returns):
    return np.sqrt(np.sum(log_returns**2))


# Function to calculate number of unique values
def unique_len(series):
    return len(series.unique())


# Function to calculate Bollinger bands
def upper_bb(prices):
    mean = prices.mean()
    std = prices.std()
    return mean + std
def middle_bb(prices):
    return prices.mean()
def lower_bb(prices):
    mean = prices.mean()
    std = prices.std()
    return mean - std


# Functions to calculate Donchian Channels
def upper_dc(prices):
    return prices.max()
def lower_dc(prices):
    return prices.min()
def middle_dc(prices):
    max_p = prices.max()
    min_p = prices.min()
    return (max_p-min_p)/2


# Function to calculate Ichimoku Clouds
def ichimoku(prices):
    cl = (1200 - prices.max() - prices.min())/2
    bl = (2 - prices.max() - prices.min())/2
    lsa = (cl + bl)/2
    lsb = (4 - prices.max() - prices.min())/2
    return lsa - lsb

In [ ]:
def calc_window_features(df, nWindows, is_book=True):
    """Aggreagate window statistics for book and trade data with different
        window sizes."""
    if is_book:
        agg_dict = {
            "wap1": ["mean", "sum", "std"],
            "wap2": ["mean", "sum", "std"],
            "spread": ["mean", "std", "sum"],
            "bid_spread": ["mean", "std", "sum"],
            'ask_spread': ["mean", "std", 'sum'],
            "log_return1": ["mean", "std", realized_volatility, 'sum'],
            "log_return2": ["mean", "std", realized_volatility, "sum"],
            "total_volume": ["mean", "sum", "std"],
            "volume_imbalance": ["mean", "std", "sum"]
        }
    
    else:
        agg_dict = {
            "price": [upper_bb, middle_bb, lower_bb, upper_dc, middle_dc, lower_dc, ichimoku, realized_volatility],
            "size" : ["mean", "std", "sum"],
            "order_count": ["median"],
            "seconds_in_bucket": [unique_len]
        }
    
    df_main = df.groupby("time_id").agg(agg_dict).reset_index()
    # rename columns to avoid multi-level indexing
    df_main.columns = ["_".join(col) for col in df_main.columns]
    for i in np.linspace(0, 599, nWindows+1)[1:-1]:
        df_grp = df[df['seconds_in_bucket'] >= i].groupby("time_id").agg(agg_dict).reset_index()
        df_grp.columns = ["_".join(col)+f"_{int(i)}" for col in df_grp.columns]
        # merge with the main dataframe
        df_main = df_main.merge(df_grp, how="left", left_on="time_id_", right_on=f"time_id__{int(i)}")
        # remove unnecessary time_id columns
        df_main = df_main.drop(f"time_id__{int(i)}", axis=1)
    
    return df_main

In [ ]:
def preprocess_book(stock_id, nWindows, is_train=True):
    """Preprocess book data one stock id at a time"""
    
    if is_train:
        df = pd.read_parquet(base_dir + f"book_train.parquet/stock_id={stock_id}")
    else:
        df = pd.read_parquet(base_dir + f"book_test.parquet/stock_id={stock_id}")
    # Extract basic granular features
    df['wap1'] = calc_wap(df['bid_price1'], df['ask_price1'], df['bid_size1'], df['ask_size1'])
    df['wap2'] = calc_wap(df['bid_price2'], df['ask_price2'], df['bid_size2'], df['ask_size2'])
    df['spread'] = (df['ask_price1'] - df['bid_price1']) + (df['ask_price2'] - df['bid_price2'])
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price2'] - df['ask_price1']
    df['log_return1'] = calc_lr(df['wap1'])
    df['log_return2'] = calc_lr(df['wap2'])
    df['total_volume'] = df['bid_size1'] + df['bid_size2'] + df['ask_size1'] + df['ask_size2']
    df['volume_imbalance'] = (df['ask_size1']+df['ask_size2'])-(df["bid_size1"]+df['bid_size2'])
    
    # Calculate window statistics
    df = calc_window_features(df, nWindows, is_book=True)
    
    # create row_id colume to facilitate merging
    df['row_id'] = str(stock_id) + "-" + df['time_id_'].astype("str")
    
    # remove time_id column to avoid redundancy while merging
    df = df.drop("time_id_", axis=1)
    
    return df


def preprocess_trade(stock_id, nWindows, is_train=True):
    """Preprocess trade data one stokc id at a time"""
    
    if is_train:
        df = pd.read_parquet(base_dir + f"trade_train.parquet/stock_id={stock_id}")
    else:
        df = pd.read_parquet(base_dir + f"trade_test.parquet/stock_id={stock_id}")
    
    # Add granular features here
    
    # Calculate window statistics
    df = calc_window_features(df, nWindows, is_book=False)
    
    # create row_id colume to facilitate merging
    df['row_id'] = str(stock_id) + "-" + df['time_id_'].astype("str")
    
    # remove time_id column to avoid redundancy while merging
    df = df.drop("time_id_", axis=1)
    
    return df

In [ ]:
def preprocess(stock_id_list, nWindows, is_train=True):
    """Parallel preprocessing of book and trade data"""
    
    # Parallel processing loop function
    def for_joblib(stock_id):
        
        # Preprocess book and trade data and merge them
        df_per_stock = pd.merge(preprocess_book(stock_id,nWindows,is_train),
                               preprocess_trade(stock_id, nWindows,is_train),
                               on="row_id",
                               how="left")
        return df_per_stock
    
    # Parallel API
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in stock_id_list)
    # Concatenate all stock's feature data together
    df = pd.concat(df, ignore_index=True)
    
    return df

## Training

In [ ]:
def stock_and_time_stats(df):
    """Calculate group statistics for volatility columns per stock_id and per time_id"""
    
    volatility_cols = df.filter(regex="_realized_volatility").columns
    
    # Group by stock id
    df_stock_id = df.groupby("stock_id")[volatility_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by time id
    df_time_id = df.groupby(['time_id'])[volatility_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')

    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    # Drop redundant columns
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)

    return df

In [ ]:
def apply_kmeans(train, test=None):
    """Apply kmeans clustering to model stocks in different sectors"""
    
    train_vol = train.filter(regex="_realized_volatility")
    train_vol['stock_id'] = train['stock_id']
    # test_vol = test.filter(regex="_realized_volatility")
    # test_vol['stock_id'] = test['stock_id']
    
    # scale the data
    scaler = MaxAbsScaler()
    train_scaled = scaler.fit_transform(pd.get_dummies(train_vol, columns=["stock_id"]))
    print(np.sum(np.isnan(train_scaled)))
    train_scaled = np.nan_to_num(train_scaled)
    # test_scaled = scaler.transform(pd.get_dummies(test_vol, columns=['stock_id']))
    # Test data have some nan values (only 0.001% of the total data)
    # test_scaled = np.nan_to_num(test_scaled)

    
    # Apply kmeans
    kmean = MiniBatchKMeans(n_clusters=11, batch_size=128)
    
    # Predict sectors
    train_sectors = kmean.fit_predict(train_scaled)
    # test_sectors = kmean.predict(test_scaled)
    train['sector'] = train_sectors
    # test['sector'] = test_sectors
    
    # Group by sector
    volatility_cols = train_vol.columns
    train_sector = train.groupby("sector")[volatility_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # test_sector = test.groupby("sector")[volatility_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    # Rename columns joining suffix
    train_sector.columns = ['_'.join(col) for col in train_sector.columns]
    train_sector = train_sector.add_suffix('_' + 'sector')
    
    # test_sector.columns = ['_'.join(col) for col in test_sector.columns]
    # test_sector = test_sector.add_suffix('_' + 'sector')
    
    # Merge with original dataframe
    train = train.merge(train_sector, how = 'left', left_on = ['sector'], right_on = ['sector__sector'])
    # test = test.merge(test_sector, how = 'left', left_on = ['sector'], right_on = ['sector__sector'])
    
    # Drop redundant columns
    train.drop(['sector__sector'], axis = 1, inplace = True)
    # test.drop(['sector__sector'], axis = 1, inplace = True)
    
    return train# , test

In [ ]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False


def train_and_evaluate(train, test, lgb_params, FOLDS, predict=True):
    
    # Prepare Input Data
    groups = train['time_id']
    x = train.drop(["row_id", "time_id", "target"], axis= 1)
    y = train['target']
    x_test = test.drop(["row_id", "time_id"], axis = 1)
    
    # Transform stock id to numeric column
    x['stock_id'] = x['stock_id'].astype("int")
    x_test['stock_id'] = x_test['stock_id'].astype("int")
    
    # Prediction arrays
    val_pred = np.zeros(x.shape[0])
    test_pred = np.zeros(x_test.shape[0])
    train_rmspe = 0
    
    # K-fold cross valiation
    kfold = GroupKFold(n_splits=FOLDS)
    for fold, (train_idx, val_idx) in enumerate(kfold.split(x, y, groups)):
        print("#"*40)
        print("\nTraining Fold - ", fold+1)
        X_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val     = x.iloc[val_idx]  , y.iloc[val_idx]
        
        # Add weights here
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        
        # create lgb datasets
        train_data = lgb.Dataset(X_train, y_train, weight = train_weights, categorical_feature = ["stock_id", "sector"])
        val_data   = lgb.Dataset(X_val,   y_val, weight = val_weights,   categorical_feature = ["stock_id", "sector"])
        
        # Train model
        model = lgb.train(params = lgb_params,
                         train_set = train_data,
                         valid_sets = [train_data, val_data],
                         num_boost_round = 9999999,
                         early_stopping_rounds = 70,
                         feval = feval_rmspe,
                         valid_names=["training", "validation"],
                         # callbacks=[neptune_callback],
                         verbose_eval = 200)
        
        # Plot feature importance
        plot_importance(model, max_num_features=20, figsize= (12, 6))
        plt.show()
        
        # Predict on train set
        train_rmspe += rmspe(y_train, model.predict(X_train)) / FOLDS
        # Predict validation set
        val_pred[val_idx] = model.predict(X_val)
        # Predict test set
        test_pred += model.predict(x_test) / FOLDS
        
    return val_pred, test_pred, train_rmspe

In [ ]:
# !pip install neptune-client
# !pip install neptune-lightgbm
# !pip install psutil

# import neptune.new as neptune
# from neptune.new.integrations.lightgbm import NeptuneCallback
# from neptune.new.types import File

# # Neptune Setup
# # Create run
# run = neptune.init(
#     project="amansaini150/Optiver",
#     api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmMTFkNjI2MS04ZTg0LTQ3YTMtYjNiZi02OGMzNmQ3ODQyZDMifQ==",
#     tags=["lgbm-integration", "GroupKFold", "reg", "group-stats", "weights", "kmeans"]
# )
# # Create neptune callback
# neptune_callback = NeptuneCallback(run=run)

# # Pass Neptune in the callback

In [ ]:
# Training and Testing

# Parameters
nWindows = 4
FOLDS = 5
lgb_params = {
        'learning_rate': 0.13572437900113307,        
        'lambda_l1': 2.154360665259325,
        'lambda_l2': 6.711089761523827,
        'num_leaves': 769,
        'min_sum_hessian_in_leaf': 20.44437160769411,
        'feature_fraction': 0.7921473067441019,
        'feature_fraction_bynode': 0.8083803860191322,
        'bagging_fraction': 0.9726755660563261,
        'bagging_freq': 42,
        'min_data_in_leaf': 690,
        'max_depth': 3,
        'seed': seed,
        'feature_fraction_seed': seed,
        'bagging_seed': seed,
        'drop_seed': seed,
        'data_random_seed': seed,
        'objective': 'rmse',
        'boosting': 'gbdt',
        'verbosity': -1,
        'n_jobs': -1,
}

# Load target datasets
train = pd.read_csv(base_dir + "train.csv")
test = pd.read_csv(base_dir + "test.csv")

# Preprocess feature data for train
train['row_id'] = train['stock_id'].astype("str") + "-" + train['time_id'].astype("str")
train_feature = preprocess(train['stock_id'].unique(), nWindows, is_train=True)
train = train.merge(train_feature, on="row_id", how="left")

# Preprocess feature data for test
test_feature = preprocess(test['stock_id'].unique(), nWindows, is_train=False)
test = test.merge(test_feature, on="row_id", how="left")

# Calculate group statistics
train = stock_and_time_stats(train)
test = stock_and_time_stats(test)

In [ ]:
# test[['row_id']].to_csv('submission.csv',index = False)

In [ ]:
with open("../input/optiverdatasetwithoutkmeans/training_data.pkl", "rb") as f:
    train = pickle.load(f)

In [ ]:
# Apply Kmeans
# train, test = apply_kmeans(train, test)
train = apply_kmeans(train)

In [ ]:
# Save training and testing data for Tuning
with open("training_data.pkl", "wb") as f:
    pickle.dump(train, f)
# with open("testing_data.pkl", "wb") as f:
#     pickle.dump(test, f)
# Load saved data
# with open("../input/optiver-processed-data/training_data.pkl", "rb") as f:
#     train = pickle.load(f)
# with open("../input/optiver-processed-data/testing_data.pkl", "rb") as f:
#     test = pickle.load(f)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.barplot(x = train['sector'].value_counts().index, y=train['sector'].value_counts())

In [ ]:
# Train and Predict
val_pred, test_pred, train_rmspe = train_and_evaluate(train, test, lgb_params, FOLDS)

In [ ]:
# # log parameters into Neptune
# run["parameters"] = lgb_params
# run["parameters/folds"] = FOLDS
# run["parameters/nWindows"] = nWindows
# run["oof_score"] = rmspe(train['target'], val_pred)
# run["train_score"] = train_rmspe
# run.stop()

In [ ]:
# # Prepare submission
test['target'] = test_pred
test[['row_id', 'target']].to_csv('submission.csv',index = False)